In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10662
2,Huelva,Confirmados PDIA 14 días,1028
3,Huelva,Tasa PDIA 14 días,"201,27539682384293"
4,Huelva,Confirmados PDIA 7 días,432
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,6
629,Municipio de Huelva sin especificar,Total Confirmados,128
630,Municipio de Huelva sin especificar,Curados,99


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10662.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3006.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1131 personas en los últimos 7 días 

Un positivo PCR cada 490 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10662.0,432.0,1028.0,510743.0,84.582657,201.275397,266.0
Huelva-Costa,5966.0,261.0,657.0,288115.0,90.588827,228.033945,156.0
Condado-Campiña,3291.0,149.0,317.0,155057.0,96.093695,204.440948,92.0
Huelva (capital),3006.0,127.0,293.0,143663.0,88.401328,203.949521,74.0
Lepe,940.0,39.0,95.0,27431.0,142.174912,346.323503,28.0
Palma del Condado (La),517.0,42.0,84.0,10761.0,390.298299,780.596599,25.0
Moguer,359.0,23.0,62.0,22088.0,104.128939,280.695400,20.0
Cartaya,370.0,22.0,63.0,19974.0,110.143186,315.410033,18.0
Isla Cristina,405.0,30.0,108.0,21264.0,141.083521,507.900677,17.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),517.0,42.0,84.0,10761.0,390.298299,780.596599,25.0
Lucena del Puerto,93.0,11.0,19.0,3371.0,326.312667,563.630970,10.0
Isla Cristina,405.0,30.0,108.0,21264.0,141.083521,507.900677,17.0
San Bartolomé de la Torre,45.0,1.0,14.0,3736.0,26.766595,374.732334,1.0
Lepe,940.0,39.0,95.0,27431.0,142.174912,346.323503,28.0
Paterna del Campo,64.0,4.0,12.0,3509.0,113.992590,341.977771,3.0
Cortelazor,2.0,1.0,1.0,299.0,334.448161,334.448161,0.0
Cartaya,370.0,22.0,63.0,19974.0,110.143186,315.410033,18.0
Jabugo,41.0,6.0,7.0,2250.0,266.666667,311.111111,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,45.0,1.0,14.0,3736.0,26.766595,374.732334,1.0,0.071429
Villalba del Alcor,103.0,1.0,10.0,3338.0,29.958059,299.580587,0.0,0.100000
Gibraleón,351.0,2.0,13.0,12607.0,15.864202,103.117316,0.0,0.153846
Valverde del Camino,273.0,2.0,11.0,12820.0,15.600624,85.803432,2.0,0.181818
Beas,57.0,1.0,4.0,4257.0,23.490721,93.962885,0.0,0.250000
Rociana del Condado,116.0,2.0,8.0,7855.0,25.461489,101.845958,0.0,0.250000
Minas de Riotinto,67.0,1.0,4.0,3848.0,25.987526,103.950104,1.0,0.250000
Isla Cristina,405.0,30.0,108.0,21264.0,141.083521,507.900677,17.0,0.277778
Punta Umbría,231.0,5.0,18.0,15242.0,32.804094,118.094738,2.0,0.277778
